# Import everything

In [1]:
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
from scipy import fftpack
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
import math
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from keras_tqdm import TQDMNotebookCallback
from tensorboard.plugins.hparams import api as hp
from livelossplot.tf_keras import PlotLossesCallback

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave
from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
import timeit
from skimage.transform import resize
from timeit import default_timer as timer
from datetime import timedelta
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.keras.models import Model, load_model
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import plot_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import bayes_mixed_glm as glm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from spectrum import arburg, arma2psd, pburg
import pylab
from scipy.signal import find_peaks, butter
from scipy.integrate import simps
from scipy.io import loadmat
from numpy import trapz

Using TensorFlow backend.
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


# Prepare paths of EEG files

In [2]:
paths_clean_hjorth = [
    'data/dataset1/original/sub02/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub02/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub02/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub03/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub03/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub03/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub04/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub04/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub04/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub05/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub05/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub05/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub06/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub06/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub06/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub07/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub07/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub07/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub08/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub08/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub08/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub10/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub10/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub10/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub12/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub12/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub12/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub13/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub13/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub13/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub15/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub15/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub15/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub16/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub16/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub16/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub17/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub17/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub17/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub18/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub18/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub18/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub19/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub19/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub19/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub20/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub20/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub20/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub21/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub21/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub21/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub22/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub22/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub22/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat'
]

In [3]:
paths = []
for path in paths_clean_hjorth:
    paths.append({
        'clean_hjorth': path,
        'clean_raw': path[0:-31] + '06-clean-prestimulus.p',
        'hjorth': path[0:-31] + '010-raw-hjorth.mat',
        'raw': path[0:-31] + 'raw.p',
    })
df_paths = pd.DataFrame(paths)

# Basic I/O and conversions.

In [4]:
def read_trials_from_mat(filename):
    x = loadmat(filename)
    mat_trials = x['dat'][0][0][3][0]
    trials = []
    time = np.linspace(-1000, 1000, len(mat_trials[0][0]))
    for mat_trial in mat_trials:
        trials.append(pd.DataFrame({'C3': mat_trial[0], 'C4': mat_trial[1], 'time': time}))
    return trials

def crop_mep_region(mep_frame, crop_start=0.211, crop_end=0.4):
    multiplier = 7499 / 1.4998
    start = int(crop_start * multiplier)
    end = int(crop_end * multiplier)
    mep_cropped = mep_frame.iloc[start:end, :]
    time = mep_frame.iloc[start:end, :]
    return mep_cropped

def get_apb_column_name(mep_frame):
    if 'L APB' in mep_frame:
        return 'L APB'
    else:
        return 'APB'

def open_mep_as_df(path):
    fileMep = open(path, "r+")
    mep_frames = fileMep.read().split('\n\n')
    df_list = []
    for mep_frame in mep_frames:
        df_list.append(pd.read_csv(StringIO(mep_frame), '\t'))
    return df_list

def get_mep_path(eeg_path):
    segments = eeg_path.split('/')
    segments[5] = 'mep'
    segments = segments[:-1]
    mep_path = '/'.join(segments) + '/*.txt'
    mep_path = glob.glob(mep_path)[0]
    return mep_path

def calculate_mep_size(mep_frame, latency, duration):
    mep_cropped = crop_mep_region(mep_frame, latency, latency + duration)
    apb_name = get_apb_column_name(mep_frame)
    max_row = mep_frame.iloc[mep_cropped.idxmax(axis=0)[apb_name]]
    min_row = mep_frame.iloc[mep_cropped.idxmin(axis=0)[apb_name]]
    mep_size = max_row[apb_name] - min_row[apb_name]
    return mep_size, min_row, max_row

def read_latency_duration(eeg_path):
    segments = eeg_path.split('/')
    sub = segments[3]
    exp = segments[4]
    run = segments[6]
    path = 'data/dataset1/original/' + sub + '/' + exp + '/mep/' + run + '/01-ld.csv'
    files = glob.glob(path)
    if len(files) < 1:
        return False
    fileMep = open(path, "r+")
    lines = fileMep.read().split('\n')
    df_ld = []
    for frame_txt in lines:
        if 'Frame' in frame_txt:
            continue
        segments = frame_txt.split('\t')
        if len(segments) >= 4:
            df_ld.append({
                'trial_num': int(segments[0])-1,
                'latency': float(segments[2]),
                'duration': float(segments[3]) - float(segments[2])
            })
    return pd.DataFrame(df_ld)

def calculate_mep_area(mep_frame, latency, duration, plot=False):
    apb_name = get_apb_column_name(mep_frame)
    mep_frame = mep_frame[(mep_frame['s'] >= latency) * (mep_frame['s'] <= latency + duration)]
    amplitude = mep_frame[apb_name]
    amplitude = np.abs(amplitude)
    time_diff = mep_frame['s'].iloc[1] - mep_frame['s'].iloc[0]
    area = trapz(amplitude, dx=time_diff)
    if (plot == True):
        plt.plot(mep_frame['s'], amplitude)
    return area

# Power and phase calculations

In [5]:
def blackman_harris_filter(channel, time, cutoffs, fs, numtaps=801):
    b = signal.firwin(numtaps, cutoffs, window='blackmanharris', fs=fs)
    filtered = signal.lfilter(b, 1, channel)
    delay = 0.5 * (numtaps - 1) / fs
    df = pd.DataFrame({
        'time': time-delay,
        'channel': filtered
    })
    return df

def butter_bandpass_filter(data, lowcut, highcut, fs, btype='bandpass', order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype=btype)
    y = signal.lfilter(b, a, data)
    return y

def get_phase(channel, time, band, fs, filter_type='butter', start_time_ms=-750, stop_time_ms=-2):
    if filter_type=='butter':
        df_filtered = pd.DataFrame({'channel': butter_bandpass_filter(channel, band[0], band[1], fs), 'time': time})
    else:
        df_filtered = blackman_harris_filter(channel, time, [0.000001, band[0]], fs)
        df_filtered = blackman_harris_filter(df_filtered['channel'].values, df_filtered['time'].values, band[1], fs)    
    
    hilb = signal.hilbert(df_filtered[(df_filtered['time'] > start_time_ms) * (df_filtered['time'] < stop_time_ms)]['channel'])
    phase = np.angle(hilb, deg=True)
    df_phase = pd.DataFrame(phase, columns=['phase'])
    df_phase['time'] = df_filtered[(df_filtered['time'] > start_time_ms) * (df_filtered['time'] < stop_time_ms)]['time'].values
    
    return df_phase.iloc[-1]['phase'] + 180

def get_power(channel, time, fs, crop_start_millis=-150, method='pburg', filter_type='butter', resample=True):       
    # Crop.
    channel = channel[(time > crop_start_millis) * (time < -1)].values
    time = time[(time > crop_start_millis) * (time < -1)].values

    # Resample.
    if resample:
        ts = time[1] - time[0]
        secs = len(channel) * ts
        secs = secs/1000
        resampled = signal.resample(channel, int(secs*fs))
        resampled_time = np.linspace(time[0], time[-1], len(resampled))
    else:
        resampled = channel
        resampled_time = time
    
    # Filer to get rid of line noise.
    if filter_type == 'butter':
        resampled = butter_bandpass_filter(resampled, 48, 52, fs, 'bandstop')
        resampled_time = resampled_time
    else:
        df_filtered = blackman_harris_filter(resampled, resampled_time, [48, 52], fs, numtaps=101)
        resampled = df_filtered['channel']
        resampled_time = df_filtered['time']
    
    # PSD.
    try:
        if method == 'welch':
            # Welch method
            freq, power = signal.welch(resampled, fs, nfft=2*fs)
            freq_res = freq[1] - freq[0]
            df_power = pd.DataFrame({'freq': freq, 'power': 10*np.log10(power)})
        elif method == 'fft':
            # FFT method
            power = np.abs(np.fft.fft(resampled, n=2*fs))
            freq = np.linspace(0, fs, len(power))
            df_power = pd.DataFrame({'freq': freq, 'power': 10*np.log10(power)})
        elif method == 'pburg':
            # Burgs method
            p = pburg(resampled, int(len(resampled)*.5), sampling=fs, NFFT=2000)
            power = p.psd
            freq = np.linspace(0, fs, len(power))
            df_power = pd.DataFrame({'freq': freq, 'power': 10*np.log10(power)})
    except:
        return 0

    df_power = df_power[df_power['freq'] < fs/2]
    return df_power

In [6]:
# row = df_paths.iloc[10]
# trials = read_trials_from_mat(row['hjorth'])
# trial = trials[10]
# for filter_type in ['butter', 'blackmann']:
#     df_power = get_power(trial['C3'], trial['time'], 2048, method='pburg', filter_type=filter_type, resample=False)
#     df_power = df_power[df_power['freq'] < 80]
#     df_power = df_power[df_power['freq'] > 3.5]
#     plt.plot(df_power['freq'], df_power['power'])
#     plt.title(filter_type)
#     plt.show()

# Combine EEG and others

In [7]:
def process_block(path_raw, path_hjorth, path_clean_raw, path_clean_hjorth):
    powers = []
    phases = []
    for artifact_removed, eeg_type, path, fs in zip([False, False, True, True, True], ['Raw', 'Hjorth', 'Raw', 'Hjorth', 'Averaged'], [path_raw, path_hjorth, path_clean_raw, path_clean_hjorth, path_clean_raw], [4096, 4096, 2048, 2048, 2048]):
        # Read EEG file.
        sub = path.split('/')[3]
        exp = path.split('/')[4]
        run = path.split('/')[6]
        if eeg_type == 'Hjorth':
            trials = read_trials_from_mat(path)
        else:
            trials = pickle.load(open(path, "rb"))
            
        # Select channel name.
        if sub in ['sub13', 'sub02']:
            channel = 'C4'
        else:
            channel = 'C3'
            
        # Calculate average of M1 electrodes.
        if eeg_type == 'Averaged':
            new_trials = []
            for trial in trials:
                trial['C3'] = trial[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
                trial['C4'] = trial[['FC6', 'FC4', 'FC2', 'C6', 'C4', 'C2', 'CP6', 'CP4', 'CP2']].mean(axis=1)
                new_trials.append(trial)
            trials = new_trials
            
        # Read latency duration.
        df_ld = read_latency_duration(path)

        # Calculate powers and phases. And combine the results in a df list.
        for trial_num in df_ld['trial_num'].values:
            if len(trials)-1 < trial_num:
                continue
            trial = trials[trial_num]
            
            # Calculate MEP parameters.
            ld_row = df_ld[df_ld['trial_num'] == trial_num].iloc[0]
            latency = ld_row['latency']
            duration = ld_row['duration']
            mep_frame = open_mep_as_df(get_mep_path(path))[trial_num]
            mep_size, _, _ = calculate_mep_size(mep_frame, latency, duration)
            mep_area = calculate_mep_area(mep_frame, latency, duration)
            
            # PSD and phase calculation.
            for filter_name, filter_code in zip(['Blackmann-Harris', 'Butterworth'], ['butter', 'blackmann-harris']):

                # PSD Calculation.
                for resampled, resample_fs in zip([True, False], [512, fs]):
                    for time in [-150, -750]:
                        for method_name, method_code in zip(['FFT', 'Welch', 'Burg'], ['fft', 'welch', 'pburg']):
                            df_power = get_power(trial[channel], trial['time'], method=method_code, filter_type=filter_code, crop_start_millis=time, fs=resample_fs, resample=resampled)
                            for band, fc in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [[3.5, 8], [8, 12], [13, 30], [30, 80]]):
                                powers.append({
                                    'sub': sub,
                                    'exp': exp,
                                    'run': run,
                                    'trial': trial_num,
                                    'ArtifactRemoved': artifact_removed,
                                    'EEG': eeg_type,
                                    'Resampled': resampled,
                                    'Filter': filter_name,
                                    'Time': time,
                                    'Method': method_name,
                                    'Band': band,
                                    'Power': df_power[(df_power['freq'] >= fc[0]) * (df_power['freq'] < fc[1])]['power'].mean(),
                                    'mep_latency': latency,
                                    'mep_duration': duration,
                                    'mep_area': mep_area
                                })
                                
                for band, fc in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [[3.5, 8], [8, 12], [13, 30], [30, 80]]):
                    phases.append({
                        'sub': sub,
                        'exp': exp,
                        'run': run,
                        'trial': trial_num,
                        'ArtifactRemoved': artifact_removed,
                        'EEG': eeg_type,
                        'Filter': filter_name,
                        'Band': band,
                        'Phase': get_phase(trial[channel], trial['time'], [fc[0], fc[1]], filter_type=filter_code, fs=fs),
                        'mep_latency': latency,
                        'mep_duration': duration,
                        'mep_area': mep_area,
                    })
    return (powers, phases)

In [8]:
# Run analysis
paths = [list(a) for a in zip(df_paths['raw'].values, df_paths['hjorth'].values, df_paths['clean_raw'].values, df_paths['clean_hjorth'].values)]
num_cores = multiprocessing.cpu_count() - 2
results = Parallel(n_jobs=num_cores)(delayed(process_block)(path_raw, path_hjorth, path_clean_raw, path_clean_hjorth) for path_raw, path_hjorth, path_clean_raw, path_clean_hjorth in tqdm(paths))

In [9]:
df_powers = []
df_phases = []
for powers, phases in tqdm(results):
    for power in powers:
        df_powers.append(power)
    for phase in phases:
        df_phases.append(phase)

df_powers = pd.DataFrame(df_powers)
df_phases = pd.DataFrame(df_phases)

In [10]:
df_new = []
for sub in tqdm(df_powers['sub'].unique()):
    df = df_powers[df_powers['sub'] == sub]
    i = 1
    for idx, row in df.iterrows():
        row['trial_abs'] = i
        i = i + 1
        df_new.append(row)
df_powers = pd.DataFrame(df_new)
print(df_powers.shape)
df_powers.head()


(1285440, 16)


,ArtifactRemoved,Band,EEG,Filter,Method,Power,Resampled,Time,exp,mep_area,mep_duration,mep_latency,run,sub,trial,trial_abs
0,False,Theta,Raw,Blackmann-Harris,FFT,41.365676,True,-150,exp01,0.006262,0.1634,0.2212,SP 110RMT r1,sub02,0,1
1,False,Mu,Raw,Blackmann-Harris,FFT,40.135540,True,-150,exp01,0.006262,0.1634,0.2212,SP 110RMT r1,sub02,0,2
2,False,Beta,Raw,Blackmann-Harris,FFT,35.602548,True,-150,exp01,0.006262,0.1634,0.2212,SP 110RMT r1,sub02,0,3
3,False,Gamma,Raw,Blackmann-Harris,FFT,32.033988,True,-150,exp01,0.006262,0.1634,0.2212,SP 110RMT r1,sub02,0,4
4,False,Theta,Raw,Blackmann-Harris,Welch,26.367312,True,-150,exp01,0.006262,0.1634,0.2212,SP 110RMT r1,sub02,0,5


In [11]:
df_powers.to_csv('164-d1-powers.csv')
df_phases.to_csv('164-d1-phases.csv')